In [1]:
#include <iostream>
#include <string>
#include <algoviz/SVG.hpp>
#include "Rocket.hpp"
#include "Layout.hpp"
#include "Items.hpp"
#include <cstdlib>
using namespace std;

AlgoViz::clear();
SVG playground = SVG(800,400);

Rect barrier[9];
Rect barrierTwo[9];

string key = "";

float gravity = 5;

int start = 50;
int rounds = 0;
int rnd;
int rnd1;
int rnd2;
int y = 0;
int positionPressed;
int rndItemIndex = 2;
int rndYCoordinate = 200;
int tmpRounds;
int pace = 0;
int score;

bool firstTime = false;
bool crashed = false;
bool flyUp = false;
bool activeLightning = false;
bool activeSlow = false;
bool activeSpeed = false;
bool activePresent = false;

Layout layout = Layout(playground);
Rocket rocket = Rocket(playground);
Items item = Items(playground);
Text screenScore;
Text finalScore;

In [2]:
void createBarrier(Rect array[]) {  
    rnd = (rand() % 6);
     
    for (int block = 0; block < 9; block = block + 1 ) {
            // make two parts at index rnd(+1 & +2) of barrier transparent
            array[rnd].setFill(255, 255, 255, 0);
            array[rnd].setColor("transparent");
            array[rnd+1].setFill(255, 255, 255, 0);
            array[rnd+1].setColor("transparent");
            array[rnd+2].setFill(255, 255, 255, 0);
            array[rnd+2].setColor("transparent");
    
            // create blocks
            array[block] = Rect(800, 422-start, 50, 50, &playground);
            array[block].setFill(0, 0, 0, 1.0);
            array[block].setColor("transparent");
        
             // block position -> y-axis
            start += 49;
    }
    start = 50;
}

In [3]:
void resetBarrier(Rect array[], int rnd, int whichBarrier){
    // change color of previous transparent blocks to black
    array[rnd].setFill(0, 0, 0, 1.0);
    array[rnd].setColor("transparent");
    array[rnd+1].setColor("transparent");
    array[rnd+1].setFill(0, 0, 0, 1.0);
    array[rnd+2].setColor("transparent");
    array[rnd+2].setFill(0, 0, 0, 1.0);
    
    // generate new random missing block
    rnd = (rand() % 6);
    
    // which barrier to reset
    if (whichBarrier == 1) {
        rnd1 = rnd;
    } else {
        rnd2 = rnd;
    }
    
    // make two blocks transparent
    array[rnd].setFill(255, 255, 255, 0);
    array[rnd].setColor("transparent");
    array[rnd+1].setFill(255, 255, 255, 0);
    array[rnd+1].setColor("transparent");
    array[rnd+2].setFill(255, 255, 255, 0);
    array[rnd+2].setColor("transparent");     
}

In [4]:
void lightningStrike(Rect array[]) {
    // simulate random backwards movement  
    for (int block = 0; block < 9; block = block + 1 ) {
           array[block].moveTo(array[block].getX() + 450, array[block].getY()); 
    }
}

In [5]:
void checkCollision (int coordinateLowerLeftX, int coordinateUpperLeftY){
    // rocket constantly x = 200 
    // check depending of block width 
        if (coordinateLowerLeftX > 150 && coordinateLowerLeftX < 225){
            if (coordinateUpperLeftY+5 < rocket.getYCoordinate() && coordinateUpperLeftY +150 > rocket.getYCoordinate()){
                score++;
            } else {
                crashed = true;
            }
        }
    
        // check upper & lower border
        if (rocket.getYCoordinate()> 400 || rocket.getYCoordinate() < 0){
            crashed = true;
        }
        
        if (crashed == true){
            // hide barriers
            for (int block = 0; block < 9; block++ ) {
                // zwei Steine transparent stellen
                barrier[block].setFill(255, 255, 255, 0);
                barrierTwo[block].setFill(255, 255, 255, 0);
                barrier[block].setColor(255, 255, 255, 0);
                barrierTwo[block].setColor(255, 255, 255, 0);
            }
            
            item.hideCurrent();
            layout.flicker();
            layout.youLost();
            rocket.rocketDead();
            finalScore = Text("Your final score: " + to_string(score), 360 , 100, &playground);
            finalScore.setAttribute("stroke", "white");    
        }

}

In [6]:
void collision(Rect array1[], Rect array2[]){
    int coordinateLowerLeftX;
    int coordinateUpperLeftY;
    
    // which barrier is first to prove collision
    if (array1[rnd1].getX() < array2[rnd2].getX()){
        // get coordinates of transpent blocks (free space) from first barrier
        coordinateLowerLeftX = array1[rnd1].getX();
        coordinateUpperLeftY = array1[rnd1+2].getY();
        
        checkCollision(coordinateLowerLeftX,coordinateUpperLeftY);
    } else {
        // get coordinates of transpent blocks (free space) from second barrier
        coordinateLowerLeftX = array2[rnd2].getX();
        coordinateUpperLeftY = array2[rnd2+2].getY();
        
        checkCollision(coordinateLowerLeftX,coordinateUpperLeftY);
    }  
}

In [7]:
void itemImpact(int rndItemIndex){
    tmpRounds = rounds;
    
    // check item collection
    if (rndItemIndex == 0){
        activeLightning = true;
        lightningStrike(barrier);
        lightningStrike(barrierTwo);
    } else if (rndItemIndex == 1){
        activeSlow = true;
    } else if (rndItemIndex == 2){
        activeSpeed = true;
    } else if (rndItemIndex == 3){
        activePresent = true;
        score += 10;
    }
}

In [8]:
void collectItem(){
    if ((200 > item.getXCoordinate() - 30)&&(200 < item.getXCoordinate() + 30)&&(rocket.getYCoordinate() > rndYCoordinate - 30)&&(rocket.getYCoordinate() < rndYCoordinate + 30)) {
        item.hideCurrent();
        itemImpact(rndItemIndex);
    }
}

In [9]:
layout.hideWelcome();

while (crashed == false && key != "x" ) {
   key = playground.lastKey();
    
    // spawn item every 190 rounds
   if (rounds>2 && rounds%190 == 1){
        rndItemIndex = (rand() % 4);
        rndYCoordinate = (rand() % 400);
    }
    
    // create inital barrier
    if (rounds == 0) {
        createBarrier(barrier);
        rnd1 = rnd;
    }
    
    if (activePresent == true && rounds - tmpRounds > 2){
        activePresent = false;
    }
 
    if (key == "ArrowUp") {
        y = y-20;
        rocket.move(y);
        flyUp = true;
        positionPressed = rocket.getYCoordinate();
    }

    if (activeLightning == false){
        collision(barrier, barrierTwo);
    } else if (rounds - tmpRounds > 80){
        activeLightning = false;
    }
    
    collectItem();

    // rotate rocket based on gravity 
    if (gravity < 0){
        rocket.rotate(-25);
    } else {
        rocket.rotate(45);
    }
    
    // change gravity
    if (flyUp == true) {
        gravity = gravity - 20; 
        // threshold gravity    
        if (gravity < -20){
            gravity = -20;
        }
        
        y = y + gravity;
        
        rocket.move(y);
        
        // switch direction
        if((rocket.getYCoordinate() - positionPressed) < 15) {
            flyUp = false;
        }
    } else {
        // maximal gravity 
        gravity = gravity + 1.5;   
            if (gravity > 12){
                gravity = 7;
            }

        y = y + gravity;

        rocket.move(y);
    }
    
    // create second barrier when first barrier reaches 400 (x-axis)
    if (barrier[0].getX() == 400 && firstTime == false) {
        firstTime = true;
        createBarrier(barrierTwo);
        rnd2 = rnd; 
    }
    
    // reset barrier
    if (barrier[0].getX() == 0) {
        resetBarrier(barrier, rnd1, 1);
    } else if (barrierTwo[0].getX() == 0) {
        resetBarrier(barrierTwo, rnd2, 2);
    }
    
    // adjust pace
    if (activeSlow == true){
        pace = 5;
        activeSlow = false;
    } else if (activeSpeed == true){
        pace = -5;
        activeSpeed = false;
    } 
    
    // reset pace after 80 rounds
    if (rounds - tmpRounds > 80){
        pace = 0;
    }
    
    // constantly move barriers
    for (int k = 0; k < 9; k++){
       barrier[k].moveTo((barrier[k].getX() - 10+pace + 850) % 850, barrier[k].getY()); 
       barrierTwo[k].moveTo((barrierTwo[k].getX() - 10+pace + 850) % 850, barrierTwo[k].getY());
    }
    
    // dynamic background
    layout.moveBackground();
    layout.resetBackground();

    screenScore = Text("Score: " + to_string(score), 10 ,-5, &playground);

    item.moveItem(rndItemIndex,rndYCoordinate);
    
    rounds++;
}